In [29]:
import pandas as pd
import numpy as np
import hashlib
from typing import List, Dict
from pathlib import Path
from datetime import datetime
import os
import glob
import csv

In [30]:
# loading in the original dataframe

project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
search_paths = [
        os.path.join(project_root, 'data', 'raw', 'WHP_EarlyNahuatl_data_*.csv'),
    ]
    
matching_files = []
for path in search_paths:
    matching_files.extend(glob.glob(path))
    

if not matching_files:
    raise FileNotFoundError(f"""
No Nahuatl data CSV files found. 
Searched in the following locations:
{chr(10).join(search_paths)}

Project Root: {project_root}
Current Script Location: {os.path.dirname(__file__)}

Possible solutions:
1. Ensure the CSV file is in the correct directory
2. Check the exact filename
3. Verify the file extension
""")
    
latest_file = max(matching_files, key=os.path.getctime)

df = pd.read_csv(latest_file, encoding='utf-8-sig')

display(df.head())

,Ref,Headword,Orthographic Variants,Principal English Translation,Attestations from sources in English,Attestations from sources in Spanish,Alonso de Molina,Frances Karttunen,Horacio Carochi / English,Andrés de Olmos,Lockhart’s Nahuatl as Written,themes,Spanish Loanword
0,WHP-171879,acazomo.,"accaçomo, acaçomo",<p>perhaps not (adverb) (see Molina)</p>,<p>acaçomo iuhqui yez yn anoço yuhquiez = whet...,NaN,<p>Acaçomo. quiça no. Aduerbio.<br /> <bibl> A...,<p>AHCAZOMŌ perhaps not / quizá no (M). In on...,<p>àcaçomō = perhaps not<br /> <bibl>Horacio C...,NaN,NaN,NaN,No
1,WHP-171881,ayac.,aiaac,"<p>no one; nobody; or, for someone to be absen...",<p>aiaac mic in mexica = None of the Mexica di...,<p>ayac guincuiliz = no se la quite nadie (Tla...,"<p>Ayac. ninguno, o nadie o estar alguno ausen...","<p>AYĀC no one / ninguno, o nadie (M) See AH-,...","<p>ayāc = no one<br /> <bibl>Horacio Carochi, ...",NaN,"<p>no one; nobody; or, for someone to be absen...",NaN,No
2,WHP-171882,acan.,NaN,"<p>nowhere, no place (see Molina, Karttunen, L...",NaN,NaN,<p>acan. en ninguna parte o lugar. aduerbio.<b...,<p>AHCĀN nowhere / en ninguna parte o lugar (M...,"<p>àcān = nowhere<br /> <bibl>Horacio Carochi,...","<p>en ningun lugar, por, de, etc.<br /> <bibl>...",<p>ahcān = (particle) nowhere<br /> <bibl>Jame...,"Cardinal Directions, Cosmos",No
3,WHP-171883,acampa.,NaN,<p>from nowhere; in no way; neither from one p...,NaN,NaN,"<p>Acampa. de ninguna parte, o ni a vna parte ...",NaN,<p>àcāmpa = nowhere<br /> <bibl>Horacio Caroch...,NaN,NaN,"Cardinal Directions, Cosmos",No
4,WHP-171884,acatto.,"acattopa, yacatopa",<p>first (see Karttunen)</p>,"<p>YACATTO, YACATTOPA = first / primero<br /> ...",NaN,NaN,NaN,NaN,NaN,NaN,"Numbers, Math",No


In [31]:
# from the describe seems like there's some duplicates across the data
print(df.describe())

               Ref Headword Orthographic Variants  \
count        31806    31806                 14408   
unique       31806    31397                 14343   
top     WHP-171879  cuacua.             descalços   
freq             1        5                     5   

                     Principal English Translation  \
count                                        24055   
unique                                       22650   
top     <p>a person's name (attested as male)</p>    
freq                                            64   

                     Attestations from sources in English  \
count                                                5868   
unique                                               5599   
top     <p>"The words '<em>-yol</em>' and '<em>-yollo<...   
freq                                                   32   

                     Attestations from sources in Spanish  \
count                                                3212   
unique                             

In [32]:
def compare_attestations(attestation_value, original_entry_value, match_type='substring'):
    """
    Compare attestation values with original entry values
    
    Args:
        attestation_value (str): Value from attestation column
        original_entry_value (str): Value from original entry column
        match_type (str): 'substring' or 'exact'
    
    Returns:
        list: Matching attestations
    """
    # handle NaN or empty values
    if pd.isna(attestation_value) or pd.isna(original_entry_value):
        return []
    
    # split attestations and strip whitespace
    attestations = [att.strip() for att in str(attestation_value).split(';') if att.strip()]
    
    # matching logic based on match type
    if match_type == 'exact':
        matching_attestations = [
            att for att in attestations 
            if att == str(original_entry_value).strip()
        ]
    else:  # substring
        matching_attestations = [
            att for att in attestations 
            if att in str(original_entry_value).strip()
        ]
    
    return matching_attestations

# define match types (you can choose)
MATCH_TYPE = 'exact'  # or 'substring'

attestation_columns = [
    'Attestations from sources in English',
    'Attestations from sources in Spanish'
]
original_entry_columns = [
    'Alonso de Molina',
    'Frances Karttunen',
    'Horacio Carochi / English',
    'Andrés de Olmos',
    'Lockhart’s Nahuatl as Written'
]
   
validation_results = {}
detailed_matching = {}

for attestation_column in attestation_columns:
    column_results = {
        'source_matching': {}
    }
    column_detailed_matching = {}
   
    # total rows in the dataset
    total_rows = len(df)
   
    # rows with non-empty attestations
    non_empty_attestations = df[df[attestation_column].notna() & (df[attestation_column] != '')]
   
    # store overall attestation statistics
    column_results['total_rows'] = total_rows
    column_results['non_empty_attestations'] = len(non_empty_attestations)
   
    # check matching for each original entry column
    for original_entry_column in original_entry_columns:
        # Create a list to store detailed matching information
        detailed_match_rows = []
        
        # Iterate through each row with non-empty attestations
        for _, row in non_empty_attestations.iterrows():
            # Compare attestations
            matching_attestations = compare_attestations(
                row[attestation_column], 
                row[original_entry_column], 
                match_type=MATCH_TYPE
            )
            
            # If any attestations match, store the details
            if matching_attestations:
                detailed_match_rows.append({
                    'Ref': row['Ref'],
                    'Headword': row['Headword'],
                    'Attestation Column': attestation_column,
                    'Original Entry Column': original_entry_column,
                    'Matching Attestations': matching_attestations,
                    'Attestation Value': row[attestation_column],
                    'Original Entry Value': row[original_entry_column]
                })
        
        # collect matching results for this source
        column_results['source_matching'][original_entry_column] = {
            'matching_attestations': len(detailed_match_rows),
            'matching_percentage': len(detailed_match_rows) / total_rows * 100
        }
        
        # store detailed matching rows
        column_detailed_matching[original_entry_column] = detailed_match_rows
   
    # calculate overall matching across all sources
    total_source_matches = sum(
        results['matching_attestations']
        for results in column_results['source_matching'].values()
    )
   
    column_results['total_source_matches'] = total_source_matches
    column_results['total_source_matching_percentage'] = total_source_matches / total_rows * 100
   
    # store results for this attestation column
    validation_results[attestation_column] = column_results
    detailed_matching[attestation_column] = column_detailed_matching

# display detailed matching
print("\nDetailed Matching:")
for attestation_col, sources in detailed_matching.items():
    print(f"\n--- {attestation_col} ---")
    for source, matches in sources.items():
        print(f"\n{source}:")
        if matches:
            for match in matches:
                print("\nMatching Row:")
                for key, value in match.items():
                    print(f"{key}: {value}")
        else:
            print("No matches found")


Detailed Matching:

--- Attestations from sources in English ---

Alonso de Molina:
No matches found

Frances Karttunen:
No matches found

Horacio Carochi / English:
No matches found

Andrés de Olmos:
No matches found

Lockhart’s Nahuatl as Written:
No matches found

--- Attestations from sources in Spanish ---

Alonso de Molina:
No matches found

Frances Karttunen:
No matches found

Horacio Carochi / English:
No matches found

Andrés de Olmos:
No matches found

Lockhart’s Nahuatl as Written:
No matches found


there's no exact matches in the attestation columns in the later sources columns, only substring matches

In [33]:
ref_numrowsduped = {}
ref_nonempty_values = {}

for index, row in df.iterrows():

    non_empty_values = {
        col: row[col] for col in original_entry_columns 
        if row[col] not in ['', np.nan, None]
    }
    
    if len(non_empty_values) > 1:
        # track the number of non-empty values
        ref_numrowsduped[row["Ref"]] = len(non_empty_values)
        
        # track the specific non-empty values
        ref_nonempty_values[row["Ref"]] = non_empty_values

print(f"Total references needing duplication: {len(ref_numrowsduped)}")


Total references needing duplication: 2873


In [34]:
new_rows = []
for index, row in df.iterrows():
    original_ref = row['Ref']
    
    # check if this reference needs duplication
    if original_ref in ref_numrowsduped:
        # get the non-empty values for this reference
        non_empty_values = ref_nonempty_values[original_ref]
        
        # add the original row first with cleared Full Original Entry
        original_row = row.copy()
        original_row['Full Original Entry'] = ''  # Clear full original entry
        original_row['Original Ref'] = original_ref  # Keep track of original reference
        new_rows.append(original_row)
        
        # create a duplicate for each non-empty value
        for col, value in non_empty_values.items():
            new_row = row.copy()
            new_row['Ref'] = f"{original_ref}_{col.replace(' ', '_').lower()}"
            new_row['Original Ref'] = original_ref
            new_row['Full Original Entry'] = value
            new_rows.append(new_row)
    else:
        # if no duplication needed, just add the row once
        row_copy = row.copy()
        row_copy['Original Ref'] = '' # Consistency: add original ref to all rows
        row_copy['Full Original Entry'] = ''    
        new_rows.append(row_copy)
        


In [ ]:
'''
 so there's an issue where some words in the headword column have prefixes that excel identifies as 
 formulas so it results to them being displayed as "NAME?", so honestly the best fix I have in mind is going through
 these two columns [Headword	Orthographic Variants] and if a cell starts with "=-" replace that with "'=-" as the "'"
 is special character that does not change the actual value or data - it's purely a formatting instruction to Excel
'''

# function to add single quote prefix to values that would be interpreted as Excel formulas
def excel_safe_text(value):
    if isinstance(value, str) and value and value.startswith('=-'):
        return "'" + value  # Add single quote prefix to prevent Excel formula interpretation
    return value

new_df = pd.DataFrame(new_rows)

# process the specific columns that need protection from Excel formula interpretation
        
# save to CSV
new_df.to_csv("WHP_EarlyNahuatl_data_rowsduplicatedv2.csv", encoding='utf-8-sig')

''' running the above function didn't really help for some reason, there were still values who had NAME? as their display
    so instead I just did a find and replace of "=-" to "'=-" in both the original WHP dataset and the interim WHP dataset
'''